# DBSCAN Clustering Notebook
Introduction
In this notebook, we will investigate the DBSCAN clustering algorithm on our dataset. Following the promising results from the Feature_Selection notebook, we will apply dimension reduction using PCA and feature selection to improve the clustering outcome.

In [3]:
import os
# Set the environment variable
os.environ['OMP_NUM_THREADS'] = '1'

import pandas as pd
from sklearn.mixture import GaussianMixture
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns


import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Load the data

In [4]:
# Read CSV files into DataFrames
df_mi_corr = pd.read_csv("../processed_data/lazy_corr_MI_10_FS.csv")
df_mi_corr_50 = pd.read_csv("../processed_data/lazy_corr_MI_50_FS.csv")
df_corr = pd.read_csv("../processed_data/lazy_corr_FS.csv")
df_mi = pd.read_csv("../processed_data/batch_corr_MI_10_FS.csv")
df = pd.read_csv("../processed_data/batch_corr_FS.csv")
df_mi_50= pd.read_csv("../processed_data/batch_corr_MI_50_FS.csv")


df_corr_scaled_mi_batch_50= pd.read_csv("../processed_data/batch_corr_batch_MI_50_FS.csv")

df_corr_scaled_nomiddle= pd.read_csv("../processed_data/batch_corr_MI_nomiddle_50_FS.csv")

df_corr_scaled_mi_nofreq_50= pd.read_csv("../processed_data/batch_corr_MI_nofreq_50_FS.csv")

## Evaluation Methods
One of the hard thing about the clustering is how to evaluate the clustering. To do so we will leverage the nature of our dataset.

weighted average of average variability of pingtime: This will reveals the how much variability within the cluster. This does not compare each cluster's quality and having more cluster is always better but this doesn't account that either.


In [5]:
def identify_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()

def average_variability_metrics(df_cluster, all_cleaned_df):
    """
    Calculate the average number of outliers and average standard deviation of ping time for each cluster,
    and the weighted averages of these values.

    Parameters:
    df_cluster (DataFrame): The DataFrame containing sensor ID and their respective clusters.
    all_cleaned_df (DataFrame): The DataFrame containing the cleaned data.

    Returns:
    DataFrame: A DataFrame with columns for cluster, average number of outliers, and average std ping time.
    float: The weighted average of the average number of outliers across clusters.
    float: The weighted average of the average standard deviation of ping time across clusters.
    """

    # List to store the results
    results = []

    # Iterate over each cluster
    for target in df_cluster["cluster"].unique():
        # Get the sensor IDs for the current cluster
        cluster_sensors = df_cluster[df_cluster["cluster"] == target]["Sensor ID"].unique()

        # Filter the all_cleaned_df for the current cluster sensors
        all_cleaned_df_target = all_cleaned_df[all_cleaned_df['Sensor ID'].isin(cluster_sensors)]

        # Group by 'Delay (us)' and 'Range (cm)', then calculate the number of outliers in 'Ping Time (us)'
        grouped_outliers = all_cleaned_df_target.groupby(['Delay (us)', 'Range (cm)'])['Ping Time (us)'].apply(identify_outliers).reset_index(name='outliers')
        
        # Calculate the average number of outliers for the current cluster
        avg_count_outliers = grouped_outliers['outliers'].mean()
        max_count_outliers = grouped_outliers['outliers'].max()

        # Group by sensor ID, delay, and range, then calculate the standard deviation of ping time
        grouped_std = all_cleaned_df_target.groupby(['Sensor ID', 'Delay (us)', 'Range (cm)']).agg(
            std_ping_time=('Ping Time (us)', 'std')
        ).reset_index()

        # Calculate the average std ping time for the current cluster
        avg_std_ping_time = grouped_std['std_ping_time'].mean()

        # Append the results to the list
        results.append({'cluster': target, 'max_count_outliers':max_count_outliers, 'avg_count_outliers': avg_count_outliers, 'avg_std_ping_time': avg_std_ping_time, 'count': len(cluster_sensors)})
    
    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)
    
    # Calculate the weighted average of the average number of outliers

    results_df['weighted_avg_count_outliers'] =(results_df['max_count_outliers']-results_df['avg_count_outliers']) /results_df['count']
    weighted_avg_count_outliers_score = results_df['weighted_avg_count_outliers'].mean()

    # Calculate the weighted average of the average std ping time
    results_df['weighted_avg_std_ping_time'] = results_df['avg_std_ping_time'] /results_df['count']*(results_df['cluster']+1)
    weighted_avg_std_ping_time_score = results_df['weighted_avg_std_ping_time'].mean()

    return results_df, weighted_avg_count_outliers_score, weighted_avg_std_ping_time_score



### Visual Investigate Clustering performance

In [6]:
def visualize_lineplot_ping_time_with_variability(df, target = []):
    """
    Visualize the effect of range on ping time for each delay separately with variability.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    """
    # Group by sensor ID, delay, and range, then calculate the mean and standard deviation of ping time
    grouped_df = df.groupby(['Sensor ID', 'Delay (us)', 'Range (cm)']).agg(
        mean_ping_time=('Ping Time (us)', 'mean'),
        std_ping_time=('Ping Time (us)', 'std')
    ).reset_index()

    target_df = grouped_df[grouped_df['Sensor ID'].isin(target)]
    # Get unique delays
    unique_delays = target_df['Delay (us)'].unique()

    for delay in unique_delays:
        subset_df = target_df[target_df['Delay (us)'] == delay]

        fig = px.line(

        )

        # Adding error bars
        for sensor_id in subset_df['Sensor ID'].unique():
            sensor_data = subset_df[subset_df['Sensor ID'] == sensor_id]
            fig.add_trace(
                go.Scatter(
                    x=sensor_data['Range (cm)'],
                    y=sensor_data['mean_ping_time'],
                    mode='lines+markers',
                    name=f'Sensor {sensor_id}',
                    error_y=dict(
                        type='data',
                        array=sensor_data['std_ping_time'],
                        visible=True
                    )
                )
            )

        # Plot reference line
        ranges = np.linspace(subset_df['Range (cm)'].min(), subset_df['Range (cm)'].max(), 100)
        reference_ping_times = 57 * ranges
        fig.add_trace(
            go.Scatter(
                x=ranges,
                y=reference_ping_times,
                mode='lines',
                line=dict(color='red', dash='dash'),
                name='Reference Line'
            )
        )

        fig.update_layout(
            xaxis_title='Range (cm)',
            yaxis_title='Mean Ping Time (us)',
            legend_title='Sensor ID',
            template='plotly_white'
        )

        fig.show()

def visualize_lineplot_ping_time_with_variability_simple(df, target=[]):
    """
    Visualize the effect of range on ping time for each delay separately with variability.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    target (list): List of target sensor IDs to visualize.
    """
    # Group by sensor ID, delay, and range, then calculate the mean and standard deviation of ping time
    grouped_df = df.groupby(['Sensor ID', 'Delay (us)', 'Range (cm)']).agg(
        mean_ping_time=('Ping Time (us)', 'mean'),
        std_ping_time=('Ping Time (us)', 'std')
    ).reset_index()

    target_df = grouped_df[grouped_df['Sensor ID'].isin(target)]
    # Get unique delays
    unique_delays = target_df['Delay (us)'].unique()

    # Define the number of columns for subplots
    num_columns = 2  # 2 columns for the grid
    num_rows = 3  # 3 rows for the grid

    # Create subplots
    fig = make_subplots(rows=num_rows, cols=num_columns, subplot_titles=[f'Delay: {delay} us' for delay in unique_delays])

    for i, delay in enumerate(unique_delays):
        subset_df = target_df[target_df['Delay (us)'] == delay]

        # Get the row and column position for the subplot
        row = (i // num_columns) + 1
        col = (i % num_columns) + 1

        # Adding error bars
        for sensor_id in subset_df['Sensor ID'].unique():
            sensor_data = subset_df[subset_df['Sensor ID'] == sensor_id]
            fig.add_trace(
                go.Scatter(
                    x=sensor_data['Range (cm)'],
                    y=sensor_data['mean_ping_time'],
                    mode='lines+markers',
                    name=f'Sensor {sensor_id}',
                    error_y=dict(
                        type='data',
                        array=sensor_data['std_ping_time'],
                        visible=True
                    )
                ),
                row=row, col=col
            )

        # Plot reference line
        ranges = np.linspace(subset_df['Range (cm)'].min(), subset_df['Range (cm)'].max(), 100)
        reference_ping_times = 57 * ranges
        fig.add_trace(
            go.Scatter(
                x=ranges,
                y=reference_ping_times,
                mode='lines',
                line=dict(color='red', dash='dash'),
                name='Reference Line'
            ),
            row=row, col=col
        )

    fig.update_layout(
        height=1500,  # Adjust height for the grid layout
        width=1500,  # Adjust width for the grid layout
        showlegend=False,
        title_text="Ping Time vs Range for Different Delays"
    )

    fig.show()

def visualize_cluster(df,cluster = 0, simple = True):
    # Load the dataset
    file_path = '../processed_data/all_data_v4-1-1_cleaned_sensor211.csv'
    all_cleaned_df = pd.read_csv(file_path)
    all_cleaned_df = all_cleaned_df.drop("Unnamed: 0",axis=1)
    
    cluster_sensors = df[df["cluster"]==cluster]["Sensor ID"].unique()
    if simple:
        visualize_lineplot_ping_time_with_variability_simple(all_cleaned_df,cluster_sensors)
    else:
        visualize_lineplot_ping_time_with_variability(all_cleaned_df,cluster_sensors)
    

# DBSCAN

### Helper codes

In [7]:
# Load the dataset
file_path = '../processed_data/all_data_v4-1-1_cleaned_sensor211.csv'
all_cleaned_df = pd.read_csv(file_path)
all_cleaned_df = all_cleaned_df.drop("Unnamed: 0",axis=1)

In [15]:
def tune_and_visualize_dbscan(data, eps_range, min_samples_range, plot_3d=False):
    """
    Perform hyperparameter tuning for DBSCAN clustering,
    store the results in a DataFrame, visualize the results, and return the best model.
    Optionally, create an interactive 3D plot of the clustering results.

    Parameters:
    - data: pd.DataFrame - The input data for DBSCAN clustering.
    - eps_range: list or array - The range of epsilon values to try.
    - min_samples_range: list or array - The range of min_samples values to try.
    - plot_3d: bool - Whether to create an interactive 3D plot of the clustering results.

    Returns:
    - best_model: DBSCAN - The best DBSCAN model based on the silhouette score.
    - results_df: pd.DataFrame - The DataFrame containing silhouette scores for each model.
    """

    # Drop the target column if it's present
    if 'target' in data.columns:
        data = data.drop('target', axis=1)

    # Standardize the features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(data)

    # Create lists to store the results
    eps_values = []
    min_samples_values = []
    silhouette_scores = []
    n_clusters_list = []

    # Loop over all combinations of eps and min_samples
    for eps, min_samples in product(eps_range, min_samples_range):
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(features_scaled)
        # Number of clusters in labels, ignoring noise if present.
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        if n_clusters < 2:
            # Silhouette score cannot be computed with less than 2 clusters
            silhouette = np.nan
        else:
            # Exclude noise points for silhouette score
            mask = labels != -1
            silhouette = silhouette_score(features_scaled[mask], labels[mask])

        eps_values.append(eps)
        min_samples_values.append(min_samples)
        silhouette_scores.append(silhouette)
        n_clusters_list.append(n_clusters)

    # Create a DataFrame to store the results
    results_df = pd.DataFrame({
        'eps': eps_values,
        'min_samples': min_samples_values,
        'silhouette_score': silhouette_scores,
        'n_clusters': n_clusters_list
    })

    # Remove rows with NaN silhouette scores (due to less than 2 clusters)
    valid_results_df = results_df.dropna(subset=['silhouette_score'])

    if valid_results_df.empty:
        print("No valid clustering found with the given parameter ranges.")
        print("Consider adjusting the eps and min_samples ranges.")
        # Optionally, you can return the results_df for further analysis
        return None, results_df

    # Find the best model based on the highest silhouette score
    best_index = valid_results_df['silhouette_score'].idxmax()
    best_eps = valid_results_df.loc[best_index, 'eps']
    best_min_samples = valid_results_df.loc[best_index, 'min_samples']
    best_model = DBSCAN(eps=best_eps, min_samples=best_min_samples)
    best_model.fit(features_scaled)
    best_labels = best_model.labels_

    # Plot the silhouette scores as a heatmap
    pivot_table = valid_results_df.pivot('min_samples', 'eps', 'silhouette_score')
    fig = px.imshow(pivot_table, 
                    x=pivot_table.columns, 
                    y=pivot_table.index, 
                    color_continuous_scale='Viridis',
                    labels={'x': 'Epsilon', 'y': 'Min Samples', 'color': 'Silhouette Score'},
                    title='Silhouette Scores for different combinations of eps and min_samples')
    fig.show()

    # Optionally, create an interactive 3D plot of the clustering results
    if plot_3d:
        if features_scaled.shape[1] < 3:
            raise ValueError("The dataset must have at least 3 features for a 3D plot.")
        
        # Prepare data for plotting
        data_plot = pd.DataFrame(features_scaled[:, :3], columns=['Feature 1', 'Feature 2', 'Feature 3'])
        data_plot['cluster'] = best_labels.astype(str)
        data_plot['cluster'] = data_plot['cluster'].replace({'-1': 'Noise'})

        # Create 3D scatter plot
        fig_3d = px.scatter_3d(
            data_plot, 
            x='Feature 1', 
            y='Feature 2', 
            z='Feature 3', 
            color='cluster',
            title='DBSCAN Clustering Results in 3D',
            labels={'cluster': 'Cluster'}
        )
        fig_3d.show()

    return best_model, results_df


In [16]:
def train_DBSCAN(df, eps=0.5, min_samples=5, visualization_method='PCA', plot_3d=False):
    """
    Train a DBSCAN model on the given dataframe, predict clusters, and visualize the results.

    Parameters:
    df (DataFrame): The DataFrame containing the features to cluster.
    eps (float): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    min_samples (int): The number of samples in a neighborhood for a point to be considered as a core point.
    visualization_method (str): The method for visualization ('PCA' or 'TSNE').
    plot_3d (bool): Whether to generate a 3D plot. If False, a 2D plot will be generated.

    Returns:
    DataFrame: The original DataFrame with an additional column for cluster labels.
    DBSCAN: The fitted DBSCAN model.
    """
    
    # Standardize the features
    df = df.copy()
    sensor_ids = df.index if 'Sensor ID' not in df.columns else df['Sensor ID']
    features = df.drop(columns=['Sensor ID']) if 'Sensor ID' in df.columns else df
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # Fit a DBSCAN model
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan.fit(features_scaled)

    # Predict cluster labels
    cluster_labels = dbscan.labels_
    df['cluster'] = cluster_labels

    print("============ Distribution of Sensors in each Cluster ============")
    print(df.groupby(["cluster"])["Sensor ID"].count() if 'Sensor ID' in df.columns else df['cluster'].value_counts())

    # Evaluate the model using silhouette score
    # Silhouette score requires at least 2 clusters (excluding noise)
    labels_unique = np.unique(cluster_labels)
    n_clusters = len(labels_unique) - (1 if -1 in cluster_labels else 0)
    
    if n_clusters > 1:
        # Exclude noise points for silhouette score
        mask = cluster_labels != -1
        silhouette_avg = silhouette_score(features_scaled[mask], cluster_labels[mask])
        print(f"Silhouette Score (excluding noise): {silhouette_avg:.4f}")
    else:
        print("Silhouette Score cannot be computed with less than 2 clusters (excluding noise).")
    
    # Visualize the clustering results using PCA or t-SNE
    if visualization_method.upper() == 'PCA':
        n_components = 3 if plot_3d else 2
        pca = PCA(n_components=n_components)
        components = pca.fit_transform(features_scaled)
        title = '3D Visualization using PCA' if plot_3d else '2D Visualization using PCA'
    elif visualization_method.upper() == 'TSNE':
        n_components = 3 if plot_3d else 2
        tsne = TSNE(n_components=n_components, random_state=42)
        components = tsne.fit_transform(features_scaled)
        title = '3D Visualization using t-SNE' if plot_3d else '2D Visualization using t-SNE'
    else:
        raise ValueError("visualization_method should be either 'PCA' or 'TSNE'.")

    # Create a DataFrame for the components
    components_df = pd.DataFrame(components, columns=[f'Component {i+1}' for i in range(n_components)])
    components_df['cluster'] = cluster_labels.astype(str)  # Convert to string for better coloring
    components_df['Sensor ID'] = sensor_ids

    # Handle noise points in visualization
    components_df['cluster'] = components_df['cluster'].replace({'-1': 'Noise'})

    # Plot the results
    if plot_3d:
        fig = px.scatter_3d(
            components_df, 
            x='Component 1', 
            y='Component 2', 
            z='Component 3', 
            color='cluster', 
            title=title,
            hover_name='Sensor ID',  # Display Sensor ID on hover
            color_discrete_sequence=px.colors.qualitative.G10  # Set color palette
        )
    else:
        fig = px.scatter(
            components_df, 
            x='Component 1', 
            y='Component 2', 
            color='cluster', 
            title=title,
            hover_name='Sensor ID',  # Display Sensor ID on hover
            color_discrete_sequence=px.colors.qualitative.G10  # Set color palette
        )
    
    fig.show()

    return df, dbscan


In [17]:
def search_dbscan_weighted_avg(df, data, eps_range=np.linspace(0.1, 1.0, 10), min_samples=5):
    """
    Perform clustering using DBSCAN over a range of eps values, and compute weighted average metrics.

    Parameters:
    - df: DataFrame containing features to cluster.
    - data: Additional data used in average_variability_metrics function.
    - eps_range: Array-like of eps values to try.
    - min_samples: The number of samples in a neighborhood for a point to be considered as a core point.

    Returns:
    - None. Prints out the weighted average metrics for each eps value.
    """
    for eps in eps_range:
        # Standardize the features
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(df)

        # Fit a DBSCAN model
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan.fit(features_scaled)

        # Get cluster labels
        cluster_labels = dbscan.labels_
        df['cluster'] = cluster_labels

        # Compute the metrics
        results_df, weighted_avg_outliers_score, weighted_avg_std_ping_time_score = average_variability_metrics(df, data)

        # Get number of clusters (excluding noise)
        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

        # Print the weighted averages
        print(f"Weighted avg variability score when eps={eps:.2f}, n_clusters={n_clusters}: {weighted_avg_std_ping_time_score}, Outlier score: {weighted_avg_outliers_score}")


### Investigate each feature sets

#### feature set with no freq and feature selection

In [18]:
eps_range = np.linspace(0.1, 5.0, 50)  # Adjust the upper limit as needed
min_samples_range = range(2, 10)       # Keep min_samples range reasonable

# Call the function
best_model, results_df = tune_and_visualize_dbscan(df_corr_scaled_mi_nofreq_50, eps_range, min_samples_range, plot_3d=True)


TypeError: DataFrame.pivot() takes 1 positional argument but 4 were given